# PromptTemplate

# Langchain (v0.3) 의 주요 Components
https://python.langchain.com/docs/how_to/#components

1. Chat models
- Chat model 은 '메시지'를 입력받아 출력하는 형태의 언어 모델. 

1. Messages

1. Prompt templates

1. Example selectors

1. LLMs

1. Output parsers

1. Document loaders

1. Text splitters

1. Embedding models

1. Vector stores

1. Retrievers


1. Chains
https://python.langchain.com/docs/versions/migrating_chains/


1. Agents

1. Memory

1. Callbakcs


# API Key

In [2]:
import os 
from dotenv import load_dotenv

print(load_dotenv())

print(f'{os.environ['OPENAI_API_KEY'][:20]}...')  # 확인용

True
sk-proj-iKU13YeoxNgF...


In [ ]:
"""
prompts class 계층도

BasePromptTemplate --> PipelinePromptTemplate
                       StringPromptTemplate --> PromptTemplate
                                                FewShotPromptTemplate
                                                FewShotPromptWithTemplates
                       BaseChatPromptTemplate --> AutoGPTPrompt
                                                  ChatPromptTemplate --> AgentScratchPadChatPromptTemplate



BaseMessagePromptTemplate --> MessagesPlaceholder
                              BaseStringMessagePromptTemplate --> ChatMessagePromptTemplate
                                                                  HumanMessagePromptTemplate
                                                                  AIMessagePromptTemplate
                                                                  SystemMessagePromptTemplate
"""
None

# 0.PromptTemplate

In [3]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.llms.base import OpenAI
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [4]:
chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ],
)

In [5]:
# PromptTemplate
# 방법1
t = PromptTemplate.from_template("What is the capital of {country}")
t.format(country = "France")

'What is the capital of France'

In [ ]:
# 템플릿을 만드는 이점 또 한가지!
# prompt template 을 디스크에 '저장'하고 'load' 할 수 있기 떼문이다.

# 나중에 LLM 다룰때 prompt 는 매우 중요합니다.
# 대규모 프로젝트에서는 
# - 'prompt 만 만드는 팀'이 있고
# - '코딩하는 팀'이 따로 있을것이다.  
#    데이터베이스나 파일 등에 만들어 저장해 놓은 prompt 를 load 해야 할겁니다

In [6]:
# 방법2
t2 = PromptTemplate(
    template = "What is the capital of {country}",
    input_variables=['country'],  # 입력변수를 직접 명시
)
t2.format(country = "France")

'What is the capital of France'

# 1.FewShotPromptTemplate
모델에 예제(example) 추가

In [ ]:
# 모델에게 '어떻게 대답해야 하는 지에 대한 예제(example)'를 AI 모델에게 주는 것이
# prompt 를 사용해서 '어떻게 대답해야 하는지 알려주는 것'보다 훨씬 좋다

# FewShotPromptTemplate 이 하는 일이 바로 그거다!
# - 이를 통해 예제(샘플)를 형식화(포맷) 할수 있다.
# - 이런 예제들을 데이터베이스등에 저장시켜놓고 활용할수도 있다

In [7]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate

In [8]:
# ↓ 이 예제들은
# 모델이 나에게 '이런 식으로 답변해 줬으면 좋겠다' 라고 제시하는 example(예제) 들.

examples = [
  {
    "question": "What do you know about France?",

    # ↓ 원하는 형식의 답변 이다..
    "answer": """
      Here is what I know:
      Capital: Paris
      Language: French
      Food: Wine and Cheese
      Currency: Euro
      """,
  },
  {
    "question": "What do you know about Italy?",
    "answer": """
      I know this:
      Capital: Rome
      Language: Italian
      Food: Pizza and Pasta
      Currency: Euro
      """,
  },
  {
    "question": "What do you know about Greece?",
    "answer": """
      I know this:
      Capital: Athens
      Language: Greek
      Food: Souvlaki and Feta Cheese
      Currency: Euro
      """,
  },
]


In [9]:
# 만약 예제 없이 모델 호출하면?
chat.invoke("What do you know about France?")

France is a country located in Western Europe. It is known for its rich history, culture, and cuisine. The capital city is Paris, which is famous for landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral.

France is the largest country in the European Union by land area and the third-largest in Europe overall. It has a population of over 67 million people. The official language is French, and the currency is the Euro.

France is a popular tourist destination, attracting millions of visitors each year to its beautiful cities, countryside, and beaches. The country is also known for its wine and cheese production, fashion industry, and art and literature.

France has a long history of political and cultural influence, including the French Revolution, the Napoleonic Wars, and the establishment of the French colonial empire. Today, France is a member of the United Nations, NATO, and the European Union, among other international organizations.

AIMessage(content='France is a country located in Western Europe. It is known for its rich history, culture, and cuisine. The capital city is Paris, which is famous for landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral.\n\nFrance is the largest country in the European Union by land area and the third-largest in Europe overall. It has a population of over 67 million people. The official language is French, and the currency is the Euro.\n\nFrance is a popular tourist destination, attracting millions of visitors each year to its beautiful cities, countryside, and beaches. The country is also known for its wine and cheese production, fashion industry, and art and literature.\n\nFrance has a long history of political and cultural influence, including the French Revolution, the Napoleonic Wars, and the establishment of the French colonial empire. Today, France is a member of the United Nations, NATO, and the European Union, among other international organizations.',

In [10]:
example_template = """
    Human: {question}
    AI: {answer}
"""

# {question}, {answer} 는 위 example 과 동일한 key 를 사용하여 포맷 준비

In [11]:
example_prompt = PromptTemplate.from_template(example_template)

example_prompt

PromptTemplate(input_variables=['answer', 'question'], input_types={}, partial_variables={}, template='\n    Human: {question}\n    AI: {answer}\n')

In [14]:
print(example_prompt.format(**examples[2]))


    Human: What do you know about Greece?
    AI: 
      I know this:
      Capital: Athens
      Language: Greek
      Food: Souvlaki and Feta Cheese
      Currency: Euro
      



In [16]:
prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,  # 사용할 prompt
    examples=examples,  # 준비된 예제들

    # 유저의 질문을 넣어보자.
    
    # suffix=
    # 형식화된 모든 예제 마지막에 나오는 내용.
    # 그래서 한번 다 형식화가 끝난 다음, 사용자의 질문은 무엇인지 나옵니다
    suffix="Human: What do you know about {country}?",
    input_variables=['country']
)

prompt

FewShotPromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, examples=[{'question': 'What do you know about France?', 'answer': '\n      Here is what I know:\n      Capital: Paris\n      Language: French\n      Food: Wine and Cheese\n      Currency: Euro\n      '}, {'question': 'What do you know about Italy?', 'answer': '\n      I know this:\n      Capital: Rome\n      Language: Italian\n      Food: Pizza and Pasta\n      Currency: Euro\n      '}, {'question': 'What do you know about Greece?', 'answer': '\n      I know this:\n      Capital: Athens\n      Language: Greek\n      Food: Souvlaki and Feta Cheese\n      Currency: Euro\n      '}], example_prompt=PromptTemplate(input_variables=['answer', 'question'], input_types={}, partial_variables={}, template='\n    Human: {question}\n    AI: {answer}\n'), suffix='Human: What do you know about {country}?')

In [17]:
print(prompt.format(country="Germany"))


    Human: What do you know about France?
    AI: 
      Here is what I know:
      Capital: Paris
      Language: French
      Food: Wine and Cheese
      Currency: Euro
      



    Human: What do you know about Italy?
    AI: 
      I know this:
      Capital: Rome
      Language: Italian
      Food: Pizza and Pasta
      Currency: Euro
      



    Human: What do you know about Greece?
    AI: 
      I know this:
      Capital: Athens
      Language: Greek
      Food: Souvlaki and Feta Cheese
      Currency: Euro
      


Human: What do you know about Germany?


In [ ]:
# step1  example 리스트를 만들고   examples
# step2  FewShotPromptTemplate 에 전달했고 examples=
# step3  어떻게 전달한 예제들을 형식화 할지 알려주었고
# step4  마지막에 질문을 포함시켰다.  suffix, input_variables

# AI 는 우리의 예제들과 똑같은 구조, 형태로 답변하게 될겁니다


In [18]:
chain = prompt | chat

In [19]:
chain.invoke({'country': 'Germany'})

AI: 
      Here is what I know:
      Capital: Berlin
      Language: German
      Food: Bratwurst and Sauerkraut
      Currency: Euro

AIMessage(content='AI: \n      Here is what I know:\n      Capital: Berlin\n      Language: German\n      Food: Bratwurst and Sauerkraut\n      Currency: Euro', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-3.5-turbo-0125'}, id='run--5f2befde-ff90-46be-81fa-6b350139aa31-0')

In [20]:
chain.invoke({'country': 'Turkey'})

AI: 
      Here is what I know:
      Capital: Ankara
      Language: Turkish
      Food: Kebab and Baklava
      Currency: Turkish Lira

AIMessage(content='AI: \n      Here is what I know:\n      Capital: Ankara\n      Language: Turkish\n      Food: Kebab and Baklava\n      Currency: Turkish Lira', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-3.5-turbo-0125'}, id='run--666754e9-3b0f-4bcd-bb3c-54312298bd36-0')

# 2.FewShotChatMessagePromptTemplate

In [22]:
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_core.prompts.few_shot import FewShotChatMessagePromptTemplate

In [23]:
examples = [
  {
    "country": "France",

    "answer": """
      Here is what I know:
      Capital: Paris
      Language: French
      Food: Wine and Cheese
      Currency: Euro
      """,
  },
  {
    "country": "Italy",
      
    "answer": """
      I know this:
      Capital: Rome
      Language: Italian
      Food: Pizza and Pasta
      Currency: Euro
      """,
  },
  {
    "country": "Greece",
    "answer": """
      I know this:
      Capital: Athens
      Language: Greek
      Food: Souvlaki and Feta Cheese
      Currency: Euro
      """,
  },
]

In [25]:
example_prompt = ChatPromptTemplate.from_messages([
    # examples 의 key 와 동일한 이름의 변수로!
    ("human", "What do you know abount {country}?"),
    ("ai", "{answer}"),
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt = example_prompt,
    examples= examples,

    # suffix= 등은 필요없다.
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert, you give short answers."),
    example_prompt,  ## !!
    ("human", "What do you know about {country}?"), 
])



In [26]:
final_prompt.format_messages(country="Germany")

[SystemMessage(content='You are a geography expert, you give short answers.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What do you know abount France?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='\n      Here is what I know:\n      Capital: Paris\n      Language: French\n      Food: Wine and Cheese\n      Currency: Euro\n      ', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What do you know abount Italy?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='\n      I know this:\n      Capital: Rome\n      Language: Italian\n      Food: Pizza and Pasta\n      Currency: Euro\n      ', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What do you know abount Greece?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='\n      I know this:\n      Capital: Athens\n      Language: Greek\n      Food: Souvlaki and Feta Cheese\n      Currency: Euro\n      ', additional_kwargs={},

In [27]:
chain = final_prompt | chat

In [28]:
chain.invoke({'country': 'Germany'})


      I know this:
      Capital: Berlin
      Language: German
      Food: Bratwurst and Sauerkraut
      Currency: Euro

AIMessage(content='\n      I know this:\n      Capital: Berlin\n      Language: German\n      Food: Bratwurst and Sauerkraut\n      Currency: Euro', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-3.5-turbo-0125'}, id='run--faf3b42e-21dc-4c9c-9ffe-5584e2354cfd-0')

In [29]:
chain.invoke({'country': 'South Korea'})


      I know this:
      Capital: Seoul
      Language: Korean
      Food: Kimchi and Bibimbap
      Currency: South Korean Won
      

AIMessage(content='\n      I know this:\n      Capital: Seoul\n      Language: Korean\n      Food: Kimchi and Bibimbap\n      Currency: South Korean Won\n      ', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-3.5-turbo-0125'}, id='run--618fbd23-b5a6-464c-98b8-b9102c1275b6-0')

In [ ]:
#  다음에는 '예제' 를 어떻게 선택하는지에 대해 배워보자.

#  때로는 수천개의 예제를 가지고 있을텐데,  이를 모두 모델에게 줄수 없는 상황이 있을수 있다.
#    이유1) 비용이 많이 든다..  많은 텍스트 땜에.
#    이유2) '허용하는 범위' 라는게 있다 => 모~든 예제들을 모델에게 줄 수는 없다.  제한이 있다 (context window)

# 3.ExampleSelector

## LengthBasedExampleSelector

In [30]:
from langchain_core.example_selectors.length_based import LengthBasedExampleSelector

In [31]:
# LengthBasedExampleSelector 는 기본적으로
# - 예제(example) 들을 형식화 할 수 있고
# - 예제의 양이 얼마나 되는지를 확인할수 있다.

# 그러면, 사용자가 설정해 놓은 세팅값에 따라 prompt 에 알맞은 예제를 골라준다.

In [ ]:
examples = [
  {
    "country": "France",

    "answer": """
      Here is what I know:
      Capital: Paris
      Language: French
      Food: Wine and Cheese
      Currency: Euro
      """,
  },
  {
    "country": "Italy",
      
    "answer": """
      I know this:
      Capital: Rome
      Language: Italian
      Food: Pizza and Pasta
      Currency: Euro
      """,
  },
  {
    "country": "Greece",
    "answer": """
      I know this:
      Capital: Athens
      Language: Greek
      Food: Souvlaki and Feta Cheese
      Currency: Euro
      """,
  },
]

In [32]:
example_prompt = PromptTemplate.from_template("Human: {country}\nAI: {answer}")

example_prompt

PromptTemplate(input_variables=['answer', 'country'], input_types={}, partial_variables={}, template='Human: {country}\nAI: {answer}')

In [33]:
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,  # 포맷팅한 결과의 크기가 얼마나 되는지 알아야 하기에 example_prompt 도 필요
    max_length=10,  # 예제의 양을 얼마나 허용할지 지정. max_length= 를 넘어가는 example 은 제외된다.
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    # examples= 대신 example_selector= 지정.
    example_selector=example_selector,   # max_length= 값에 따라 example 의 양을 정해준다.

    suffix="Human: What do you know about {country}",
    input_variables = ['country'],
)

In [34]:
prompt.format(country='Brazil')

'Human: What do you know about Brazil'

In [ ]:
# ↑ 보다시피 선택된 예제가 없다!
# ↑ example 들이 하나도 포맷팅이 안되어 있다??
# max_length 값이 너무 작아서 exmaple 하나도 포맷팅 안됨.

# ↓ max_length= 값을 변경해보자


In [36]:
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=80,  # max_length 조정
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,   # max_length= 값에 따라 example 의 양을 정해준다.

    suffix="Human: What do you know about {country}",
    input_variables = ['country'],
)

print(prompt.format(country='Brazil'))

Human: France
AI: 
      Here is what I know:
      Capital: Paris
      Language: French
      Food: Wine and Cheese
      Currency: Euro
      

Human: What do you know about Brazil


In [38]:
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=150,  # max_length 조정
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,   # max_length= 값에 따라 example 의 양을 정해준다.

    suffix="Human: What do you know about {country}",
    input_variables = ['country'],
)

print(prompt.format(country='Brazil'))

Human: France
AI: 
      Here is what I know:
      Capital: Paris
      Language: French
      Food: Wine and Cheese
      Currency: Euro
      

Human: Italy
AI: 
      I know this:
      Capital: Rome
      Language: Italian
      Food: Pizza and Pasta
      Currency: Euro
      

Human: What do you know about Brazil


## Custom ExampleSelector (BaseExampleSelector)

In [ ]:
# ↓ 이제 자신만의 example selector 를 만드는 방법에 대해 배워보자
# 이를 통해 내가 원하는 대로의 examples 들을 얼마나 허용할지 정할수 있게 될거다.

# 가령, 애플리케이션에서 유저의 로그인 여부 또는 유저가 사용하는 언어에 따라
# 얼마나 많은 examples 들을 허용할지 정할수도 있는 것이다.

# 여기선 RandomExampleSelector 라는 이름의 ExampleSelector 를 를 '직접' 만들어 사용해 볼거다.


In [39]:
from langchain_core.example_selectors.base import BaseExampleSelector

In [40]:
# BaseExampleSelector 의 구현객체를 만드려면
#  상속 받은뒤 select_examples() 과 add_example() 을 반드시 오버라이딩 해주어야 한다.

class RandomExampleSelector(BaseExampleSelector):

    def __init__(self, examples):
        self.examples = examples

    # select_examples()
    # 입력에 따라 어떠한 샘플을 사용할지 select 함.
    
    # 이번 예제에서는 examples 리스트 에서 random 으로 선택하게 하려 함.
    # ※ 이는 얼마든지 복잡하게 만들어 볼수도 있다.
    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]  # examples 에서 무작위로 1개 선택
    
    
    # add_example()
    # Add new example to store.  이미 존재하는 example 에 example 을 추가하는 method
    def add_example(self,example):
        self.examples.append(example)
    



In [80]:
example_selector = RandomExampleSelector(examples=examples)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,  # 커스텀 ExampleSelector !

    suffix="Human: What do you know about {country}",
    input_variables = ['country'],
)

print(prompt.format(country='Brazil'))

Human: Greece
AI: 
      I know this:
      Capital: Athens
      Language: Greek
      Food: Souvlaki and Feta Cheese
      Currency: Euro
      

Human: What do you know about Brazil


# 4.PromptTemplate 저장/읽기

In [ ]:
# prompt 를 파일로 다룰 경우
#   'JSON' 파일  혹은 'YAML' 파일 로 만들수 있다

## json 파일

In [81]:
with open('prompt.json', 'w') as f:
    f.write("""
    {
      "_type": "prompt",
      "template": "What is the capital of {country}",
      "input_variables": ["country"]
    }
    """)

In [82]:
from langchain_core.prompts.loading import load_prompt

In [83]:
!dir *.json

 D 드라이브의 볼륨: 새 볼륨
 볼륨 일련 번호: 02B1-19A2

 D:\NLP2507\LangWork\LangChain기초 디렉터리

2025-12-04  오후 09:10               139 prompt.json
               1개 파일                 139 바이트
               0개 디렉터리  451,150,381,056 바이트 남음


In [84]:
prompt = load_prompt('./prompt.json')  # PromptTemplate 객체 리턴

prompt

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='What is the capital of {country}')

In [87]:
prompt.format(country='Canada')

'What is the capital of Canada'

## yaml 파일

In [ ]:
# prompt.yaml 파일을 만든다

# name: value
#  ★ name 은 쌍따옴표 없다.  : 다음에 한칸 꼭 띄우기!   뒤에 콤마 없다
"""
_type: "prompt"
"""
None

In [88]:
with open('prompt.yaml', 'w') as f:
    f.write("""
_type: "prompt"
template: "What is the capital of {country}"
input_variables: ["country"]
    """)

In [89]:
!type prompt.yaml


_type: "prompt"
template: "What is the capital of {country}"
input_variables: ["country"]
    


In [90]:
prompt = load_prompt('./prompt.yaml')
print(prompt)

prompt.format(country='England')

input_variables=['country'] input_types={} partial_variables={} template='What is the capital of {country}'


'What is the capital of England'

---

# Caching

In [91]:
# Caching 을 사용하면 모델의 응답을 저장할수 있다.
# 예를들어
# 똑같은 질문을 받는 상황이라면 그 때마다 답변생성할 필요 없이
# 이미 캐싱된 답변을 재사용 할수 있는 것이다 -->  비용절감! 

## set_llm_cache(), InMemoryCache

In [92]:
from langchain_core.globals import set_llm_cache
from langchain_core.caches import InMemoryCache

In [93]:
set_llm_cache(InMemoryCache())  # LLM 의 모든 response 는 '메모리' 에 cache 된다.

In [94]:
chat = ChatOpenAI(temperature=0.1)

In [97]:
# 동일한 질문을 여러번 해볼거다.  시간 측정하는 함수 준비
import time
from datetime import timedelta

def check_laptime(message):
    start_time = time.time()
    response = chat.invoke(message)
    end_time = time.time()
    elapsed_time = end_time - start_time # 경과시간
    print('▶ 경과시간 %s' % (str(timedelta(seconds = elapsed_time))))
    print(f'{len(response.content)} 글자: {response.content}\n')    


In [98]:
check_laptime("How do you make Italian pizza")

▶ 경과시간 0:00:05.675641
1805 글자: To make authentic Italian pizza, you will need the following ingredients:

- 2 1/4 cups of all-purpose flour
- 1 teaspoon of salt
- 1 teaspoon of sugar
- 1 packet of active dry yeast
- 1 cup of warm water
- 2 tablespoons of olive oil
- Tomato sauce
- Fresh mozzarella cheese
- Fresh basil leaves
- Any additional toppings of your choice (such as pepperoni, mushrooms, or olives)

Here is a step-by-step guide to making Italian pizza:

1. In a large mixing bowl, combine the flour, salt, and sugar. In a separate small bowl, dissolve the yeast in the warm water and let it sit for about 5 minutes until it becomes frothy.

2. Pour the yeast mixture and olive oil into the flour mixture and stir until a dough forms. Knead the dough on a floured surface for about 5-7 minutes until it becomes smooth and elastic.

3. Place the dough in a greased bowl, cover it with a clean towel, and let it rise in a warm place for about 1-2 hours until it doubles in size.

4. Preheat 

In [99]:
check_laptime("How do you make Italian pizza")

▶ 경과시간 0:00:00
1805 글자: To make authentic Italian pizza, you will need the following ingredients:

- 2 1/4 cups of all-purpose flour
- 1 teaspoon of salt
- 1 teaspoon of sugar
- 1 packet of active dry yeast
- 1 cup of warm water
- 2 tablespoons of olive oil
- Tomato sauce
- Fresh mozzarella cheese
- Fresh basil leaves
- Any additional toppings of your choice (such as pepperoni, mushrooms, or olives)

Here is a step-by-step guide to making Italian pizza:

1. In a large mixing bowl, combine the flour, salt, and sugar. In a separate small bowl, dissolve the yeast in the warm water and let it sit for about 5 minutes until it becomes frothy.

2. Pour the yeast mixture and olive oil into the flour mixture and stir until a dough forms. Knead the dough on a floured surface for about 5-7 minutes until it becomes smooth and elastic.

3. Place the dough in a greased bowl, cover it with a clean towel, and let it rise in a warm place for about 1-2 hours until it doubles in size.

4. Preheat your ov

## set_debug()

In [100]:
from langchain_core.globals import set_debug

In [101]:
set_debug(True)

In [102]:
chat.invoke("How do you make Korean Kimbab")

[llm/start] [llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make Korean Kimbab"
  ]
}
[llm/end] [llm:ChatOpenAI] [4.05s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "To make Korean kimbap, you will need the following ingredients:\n\n- 4 cups cooked short grain rice\n- 5-6 sheets of seaweed (nori)\n- 1/2 lb of bulgogi (marinated beef) or other protein of your choice\n- 1 carrot, julienned\n- 1 cucumber, julienned\n- 1/2 cup pickled radish, julienned\n- 4 eggs, beaten and cooked into thin omelettes\n- Sesame oil\n- Salt\n- Sugar\n- Soy sauce\n- Rice vinegar\

AIMessage(content='To make Korean kimbap, you will need the following ingredients:\n\n- 4 cups cooked short grain rice\n- 5-6 sheets of seaweed (nori)\n- 1/2 lb of bulgogi (marinated beef) or other protein of your choice\n- 1 carrot, julienned\n- 1 cucumber, julienned\n- 1/2 cup pickled radish, julienned\n- 4 eggs, beaten and cooked into thin omelettes\n- Sesame oil\n- Salt\n- Sugar\n- Soy sauce\n- Rice vinegar\n\nInstructions:\n\n1. Cook the rice according to package instructions and let it cool slightly.\n2. Season the rice with a mixture of sesame oil, salt, and sugar to taste.\n3. Lay a sheet of seaweed on a bamboo sushi mat or a clean kitchen towel.\n4. Spread a thin layer of rice over the seaweed, leaving a small border at the top.\n5. Arrange the bulgogi, carrot, cucumber, pickled radish, and egg omelette in a line across the rice.\n6. Roll the kimbap tightly using the sushi mat or towel, pressing gently to seal.\n7. Cut the kimbap into bite-sized pieces using a sharp knife.\n8.

In [103]:
set_debug(False)

## SQLiteCache

In [104]:
from langchain_community.cache import SQLiteCache

In [105]:
set_llm_cache(SQLiteCache('cache.db'))

In [106]:
!dir

 D 드라이브의 볼륨: 새 볼륨
 볼륨 일련 번호: 02B1-19A2

 D:\NLP2507\LangWork\LangChain기초 디렉터리

2025-12-04  오후 09:34    <DIR>          .
2025-12-04  오후 09:34    <DIR>          ..
2025-12-04  오후 07:27    <DIR>          .ipynb_checkpoints
2025-12-04  오후 07:26            68,830 01 Hello LangChain.ipynb
2025-12-04  오후 09:33            60,202 02 PromptTemplate.ipynb
2025-12-04  오후 09:34            32,768 cache.db
2025-12-04  오후 09:10               139 prompt.json
2025-12-04  오후 09:15                99 prompt.yaml
               5개 파일             162,038 바이트
               3개 디렉터리  451,150,323,712 바이트 남음


In [107]:
check_laptime("How do you make Italian pizza")

▶ 경과시간 0:00:03.396522
1872 글자: To make an authentic Italian pizza, you will need the following ingredients:

- 2 1/4 cups of all-purpose flour
- 1 teaspoon of salt
- 1 teaspoon of sugar
- 1 packet of active dry yeast
- 1 cup of warm water
- 2 tablespoons of olive oil
- Tomato sauce
- Fresh mozzarella cheese
- Fresh basil leaves
- Any additional toppings of your choice (such as pepperoni, mushrooms, or olives)

Here is a step-by-step guide to making Italian pizza:

1. In a large mixing bowl, combine the flour, salt, and sugar. In a separate small bowl, dissolve the yeast in warm water and let it sit for about 5 minutes until it becomes frothy.

2. Pour the yeast mixture and olive oil into the dry ingredients and mix until a dough forms. Knead the dough on a floured surface for about 5-7 minutes until it becomes smooth and elastic.

3. Place the dough in a lightly oiled bowl, cover it with a clean towel, and let it rise in a warm place for about 1-2 hours until it doubles in size.

4. Pr

In [108]:
check_laptime("How do you make Italian pizza")

▶ 경과시간 0:00:00.001995
1872 글자: To make an authentic Italian pizza, you will need the following ingredients:

- 2 1/4 cups of all-purpose flour
- 1 teaspoon of salt
- 1 teaspoon of sugar
- 1 packet of active dry yeast
- 1 cup of warm water
- 2 tablespoons of olive oil
- Tomato sauce
- Fresh mozzarella cheese
- Fresh basil leaves
- Any additional toppings of your choice (such as pepperoni, mushrooms, or olives)

Here is a step-by-step guide to making Italian pizza:

1. In a large mixing bowl, combine the flour, salt, and sugar. In a separate small bowl, dissolve the yeast in warm water and let it sit for about 5 minutes until it becomes frothy.

2. Pour the yeast mixture and olive oil into the dry ingredients and mix until a dough forms. Knead the dough on a floured surface for about 5-7 minutes until it becomes smooth and elastic.

3. Place the dough in a lightly oiled bowl, cover it with a clean towel, and let it rise in a warm place for about 1-2 hours until it doubles in size.

4. Pr

## debug, cache 사용 끄기

In [109]:
set_debug(False)
set_llm_cache(None)

# Model Config

In [ ]:
# 이번 단원에서 두가지를 배워볼거다

# 첫째:  OpanAI 의 모델을 사용할 때 우리가 지출하는 비용을 하는 방법
# 둘째: 모델을 어떻게 저장하고(serialize) 불러오는지에 대한 방법

## OpenAI 모델 호출 비용 확인

In [110]:
from langchain_community.callbacks.manager import get_openai_callback

In [111]:
with get_openai_callback() as usage:
    chat.invoke("What is the recipe for soju")
    print('🟦', usage)

🟦 Tokens Used: 202
	Prompt Tokens: 14
		Prompt Tokens Cached: 0
	Completion Tokens: 188
		Reasoning Tokens: 0
Successful Requests: 1
Total Cost (USD): $0.00028900000000000003


In [112]:
with get_openai_callback() as usage:
    a = chat.invoke("What is the recipe for soju")
    b = chat.invoke("What is the recipe for kimbab")
    print('\n🤧', a.content)
    print('\n🥶', b.content)
    print('🟦', usage)


🤧 Ingredients:
- 1 cup of rice
- 1 cup of water
- 1 tablespoon of yeast
- 1 tablespoon of sugar

Instructions:
1. Rinse the rice thoroughly and soak it in water for at least 1 hour.
2. Drain the rice and place it in a steamer basket. Steam the rice for about 30 minutes, or until it is cooked through.
3. Allow the rice to cool to room temperature.
4. In a large bowl, mix the yeast and sugar with the water until dissolved.
5. Add the cooked rice to the yeast mixture and stir well.
6. Cover the bowl with a clean cloth and let it sit at room temperature for 1-2 days, stirring occasionally.
7. After 1-2 days, strain the mixture through a cheesecloth or fine mesh strainer to remove any solids.
8. Transfer the liquid to a clean bottle or jar and store it in the refrigerator.
9. Serve the homemade soju chilled and enjoy responsibly.

🥶 Ingredients:
- 4 cups cooked short-grain white rice
- 5-6 sheets of seaweed (nori)
- 1 carrot, julienned
- 1 cucumber, julienned
- 1/2 red bell pepper, julienn

## 모델 config 저장하고 불러오기

In [113]:
llm = OpenAI(
    temperature=0.1,
    max_tokens=450,  # !!
    model="gpt-4-turbo" # 사용 모델 설정
)

# 모델(config)을 저장하려면
llm.save("model.json")

In [114]:
!type model.json

{
    "model_name": "gpt-4-turbo",
    "temperature": 0.1,
    "top_p": 1,
    "frequency_penalty": 0,
    "presence_penalty": 0,
    "n": 1,
    "seed": null,
    "logprobs": null,
    "max_tokens": 450,
    "_type": "openai"
}


In [115]:
from langchain_community.llms.loading import load_llm

In [116]:
llm = load_llm('model.json')

D:\NLP2507\LangWork\.venv\Lib\site-packages\langchain_community\llms\openai.py:255: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(
D:\NLP2507\LangWork\.venv\Lib\site-packages\langchain_community\llms\openai.py:1089: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(
